In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time


bounds = np.array([[-1, 1], [-1, 1], [-1, 1], [-1, 1]])
Pareto = []
colors = ["#FF0000", "#0000FF", "#000000", "#FF00FF", "#00FFFF", "#FF8000", "#800080", "#808000", "#800000", "#008080"]
points1 = []
points2 = []
sum_time = 0

def compute_gradients(points):
    gradients = []
    for i in range(1, len(points) - 1):
        gradient = (points[i + 1] - points[i - 1]) / 2  
        gradients.append(gradient)
    return np.array(gradients)

class Particle:
    def __init__(self, bounds):
        self.position = np.random.uniform(bounds[0][0], bounds[1][1], size=(4,))
        self.velocity = np.random.uniform(-1, 1, size=(4,))
        self.best_position = np.copy(self.position)
        self.best_value = self.evaluate()[0]
        self.fitness_global = self.evaluate()[1]
        self.f = 0
        
    def evaluate(self):
        f1 = (-self.position[0]**3 + 4*self.position[2]**2 + 10*self.position[3]+156.2399)/369.7137
        f2 = (self.position[0]**3 + 2*self.position[1]**2 + 10*self.position[3] + 175)/340.4638
        fitness = a1 * f1 + a2 * f2
        return np.array([f1, f2]), fitness

    def check_constraints(self):
        if (self.position[0]**2 + 2*self.position[1] + 6 * self.position[2]**3 + 24 * self.position[3] > 120 or
            6 * self.position[2]**3 + 24 * self.position[3] < -60 or
            pow(self.position[0], 2) + 3*pow(self.position[1], 2) + 5*self.position[2]**2 + pow(self.position[3], 2) > 150):
            self.f = 1
        else:
            self.f = 0
            
    def update(self, global_best_position, global_best_fitness, gradients=None):
        inertia = 0.5 * self.velocity
        cognitive = np.random.rand() * (self.best_position - self.position)
        social = np.random.rand() * (global_best_position - self.position)
        

        if gradients is not None:
            grad_effect = np.mean(gradients, axis=0)  
            self.velocity = inertia + cognitive + social + grad_effect
        else:
            self.velocity = inertia + cognitive + social
            
        self.position += self.velocity
        self.check_constraints()
        
        if self.f == 1:
            self.position -= self.velocity
        else:
            current_value = self.evaluate()[0]
            current_fitness = self.evaluate()[1]
            points1.append(current_value[0])
            points2.append(current_value[1])
            if np.all(current_fitness > self.fitness_global):
                self.best_position = np.copy(self.position)
                self.fitness_global = current_fitness
                self.best_value = current_value

def pso(num_particles, num_iterations, bounds):
    global points1, points2
    points1, points2 = [], []
    global particles 
    particles = [Particle(bounds) for _ in range(num_particles)]
    global_best_position = particles[0].best_position
    global_best_value = particles[0].best_value
    global_best_fitness = particles[0].fitness_global
    cntr = 0
    previous_best_point = [0,0]
    for iteration in range(num_iterations):
        for particle in particles:
            particle.update(global_best_position, global_best_fitness)

            if np.all(particle.fitness_global > global_best_fitness):
                global_best_position = particle.best_position
                global_best_fitness = particle.fitness_global
                global_best_value = particle.best_value
            
        plt.scatter(points1, points2, c=colors[_])
        plt.scatter(global_best_value[0], global_best_value[1], c='g')
        plt.title(f"Итерация № {iteration + 1}")
        plt.show()
        print(global_best_value)
        print(previous_best_point)
        if np.all(global_best_value == previous_best_point):
            cntr+=1
        else:
            cntr = 0
        previous_best_point = global_best_value
        if cntr == 10:
            break
    return global_best_position


for _ in range(9):

    a1 = 0.1*(_+1)
    a2 = 1 - a1
    start_time = time.time()
    
    best_solution = pso(num_particles=1000, num_iterations=50, bounds=bounds)
    

    if len(Pareto) >= 3:
        gradients = compute_gradients(np.array(Pareto))

        for particle in particles:
            particle.update(best_solution, best_solution, gradients)
    print("A1 = ",0.1*(_+1))
    print("A2 = ",1.0-0.1*(_+1))
    print("--- %s seconds ---" % (time.time() - start_time))
    print("Лучшее найденное решение:", best_solution)
    sum_time += (time.time() - start_time)
    Pareto.append(list(best_solution))

print(Pareto)
print("Среднее время: %s seconds" % (sum_time / 10))

A1 =  0.1
A2 =  0.9
--- 0.5761280059814453 seconds ---
Лучшее найденное решение: [ 4.18287104  5.26472236 -1.91079021  5.57638674]
A1 =  0.2
A2 =  0.8
--- 0.5805211067199707 seconds ---
Лучшее найденное решение: [ 0.61230909 -2.80162499 -2.59911002  9.60733599]
A1 =  0.30000000000000004
A2 =  0.7
--- 0.5559039115905762 seconds ---
Лучшее найденное решение: [-0.39035896 -4.32503159 -2.26491569  8.25110355]
A1 =  0.4
A2 =  0.6
--- 0.6656255722045898 seconds ---
Лучшее найденное решение: [-0.4823985   3.76355876  0.53243488  4.63893934]
A1 =  0.5
A2 =  0.5
--- 0.6065871715545654 seconds ---
Лучшее найденное решение: [-0.84186216 -3.47301916  0.6693631   5.18491124]
A1 =  0.6000000000000001
A2 =  0.3999999999999999
--- 0.5915665626525879 seconds ---
Лучшее найденное решение: [ 0.48784331 -1.85919124  0.7550242   5.03741374]
A1 =  0.7000000000000001
A2 =  0.29999999999999993
--- 0.63871169090271 seconds ---
Лучшее найденное решение: [ 9.99629488e-02 -5.99023218e-03 -3.60548649e+00  9.219123

: 